In [ ]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
from plotly.graph_objs import Histogram as H
import plotly.plotly as py
import pandas as pd
import numpy as np
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import datetime

In [ ]:
#Load games data
meta_games = pd.read_csv('../Data/extra_details_complete.csv')

# make new year column and replace tbd values with np.nans
meta_games['year'] = np.nan
meta_games.replace('tbd', np.nan)

# enter year value from date column
for i, row in meta_games.iterrows():
    year_value = meta_games['date'][i].split(',')[1].strip()
    meta_games.at[i,'year'] = int(year_value)

meta_games = meta_games.replace('tbd', np.nan)
meta_games_years = {}

# make a dict of meta_game dataframes for each year from 2000 to 2019
for i in range(2000,2019):
    meta_games_years[i] = meta_games[(meta_games['year'] == i)]
    
meta_games.head(2)


In [ ]:
#init some variables
years = list(range(2000, 2019))
metascore_color = "#8da0cb"
userscore_color = "#fc8d62"
#metascore_color = "#e78ac3"
#userscore_color = "#8da0cb"
new_data_list=[]
bar_opacity = 1

## make datatraces and append to datalist
for year in years:   
    year_metatrace = go.Histogram(
            visible = False,
            x=meta_games_years[year]['metascore'],
            ##ADD/REMOVE HISTNORM FOR NORMALISING
            histnorm='',
            name='metascore',
            xbins=dict(
                start=0,
                end=100,
                size=5
            ),
            marker=dict(
                color=metascore_color,
            ),
            opacity=bar_opacity
        )
    year_usertrace = go.Histogram(
            visible = False,
            x=meta_games_years[year]['userscore'].astype(float) * 10,
            ##ADD/REMOVE HISTNORM FOR NORMALISING
            histnorm='',
            name='userscore',
            xbins=dict(
                start=0,
                end=100,
                size=5
            ),
            marker=dict(
                color=userscore_color,
            ),
            opacity=bar_opacity
        )    
    new_data_list.append(year_metatrace)
    new_data_list.append(year_usertrace)
    #new_data_list.append(meta_mean) #only for mean of metascore
    
    
new_data_list[0]['visible']=True
new_data_list[1]['visible']=True
#new_data_list[2]['visible']=True #metascore mean



#list of bools for determining what trace to show as visible in 'args'
data_bools = []
for i in range(0, len(new_data_list), 2):
    temp_big_bool = len(new_data_list) * [False]
    temp_big_bool[i] = True
    temp_big_bool[i + 1] = True
    data_bools.append(temp_big_bool)

In [ ]:
#Run to make plot

#make steps for slider
steps = []
for i in range(len(meta_games_years)):
    step = dict(
        method = 'restyle',  
        args = ['visible', data_bools[i]],
        label = 2000 + i
    )
    steps.append(step)
    
#make slider for layout
sliders = [dict(
    active = 0,
    currentvalue = {"prefix": "Year: "},
    pad = {"t": 50},
    steps = steps
)]

#make layout
layout = dict(sliders=sliders,
              showlegend=True,
              width=1150,
              height=650,
              autosize=False,
              yaxis=dict(
                  range=[0, 200],
                  title='Count'
              ),
              xaxis=dict(
                  range=[0, 100],
                  title='Userscore and metascores'
              )
             )

fig = dict(data=new_data_list, layout=layout)

py.iplot(fig, filename='Metascore+userscore for 2000-2019')